<a href="https://colab.research.google.com/github/s-pike3/Projects_In_AI-ML/blob/main/HW6_part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sarah Pike \\
Projects in AI&ML \\
Homework 6, Part 3

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler  # for feature scaling
from sklearn.preprocessing import MinMaxScaler  # for feature scaling
from sklearn.metrics import confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import GridSearchCV
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 3.1 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()

Saving test1.csv to test1.csv
Saving train1.csv to train1.csv


### Prepare Data

In [3]:
traindf = pd.read_csv("train1.csv")
testdf = pd.read_csv("test1.csv")

In [4]:
class MovieLensDataset(Dataset):
    def __init__(
            self,
            df: pd.DataFrame,
            num_neg: int = 12,
            train: bool = True
        ):
        self.df = df
        self.users = df["user_id"].unique()
        self.items = df["item_id"].unique()
        self.item_set = set(self.items)
        self.user_to_idx = {user: i for i, user in enumerate(self.users)}
        self.item_to_idx = {item: i for i, item in enumerate(self.items)}
        self.num_neg = num_neg
        self.train = train
        if self.train:
          self.add_negative_samples()

    def add_negative_samples(self):
      for user in self.users:
        i = 0
        while(i < self.num_neg):
          rand_item = np.random.randint(1, 1683)
          if((rand_item in self.item_set) and ((self.df['user_id'] == user) & (self.df['item_id'] == rand_item)).any()):
            new_row = {'user_id': user, 'rating': 0}
            self.df.loc[len(self.df)] = [user,rand_item,0,0]
            i = i + 1

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        return {
            "user_id": torch.tensor(self.user_to_idx[row["user_id"]], dtype=torch.long),
            "item_id": torch.tensor(self.item_to_idx[row["item_id"]], dtype=torch.long),
            "rating": torch.tensor(row["rating"], dtype=torch.float),
        }


In [5]:
num_users = traindf.user_id.unique().shape[0]
num_items = traindf.item_id.unique().shape[0]
num_users,num_items

(943, 1650)

In [6]:
trainset = MovieLensDataset(traindf)
trainiter = DataLoader(trainset, batch_size=8, shuffle=True)

In [27]:
testset = MovieLensDataset(testdf,train=False)
testiter = DataLoader(testset, batch_size=1, shuffle=True)

### Matrix Factorization

In [8]:
# Adapted from https://eigenvalue.medium.com/a-hackers-guide-to-neural-collaborative-filtering-with-pytorch-lightning-defa99236c78
class MF(nn.Module):
  def __init__(self, num_users, num_items, embedding_dim=8):
    super(MF, self).__init__()
    self.P = nn.Embedding(num_users, embedding_dim)
    self.Q = nn.Embedding(num_items, embedding_dim)
    self.output_layer = nn.Linear(embedding_dim, 1)

  def forward(self, user_ids, item_ids):
    # Get embeddings for users and items
    p_u = self.P(user_ids)
    q_i = self.Q(item_ids)
    # Element-wise product
    gmf = torch.mul(p_u, q_i)
    # Linear layer which allows for varying importance of latent dimensions
    output = self.output_layer(gmf)
    return output

In [9]:
model = MF(num_users,num_items)
learning_rate = 0.01
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
num_epochs = 30
model.train()

for epoch in range(num_epochs):
  for i,data in enumerate(trainiter):
      user,item,rating = data['user_id'],data['item_id'],data['rating']


      # forward pass and loss

      y_predicted = model.forward(user,item)

      loss = criterion(y_predicted,rating.unsqueeze(-1))

      # backward pass
      loss.backward()

      # updates
      optimizer.step()

      # zero gradients
      optimizer.zero_grad()




#### Evaluation

In [105]:
preds = np.array([])
ratings = []
model.eval()
user_dict = dict()
top_ratings = dict()
for i,data in enumerate(testiter):
    user,item,rating = data['user_id'],data['item_id'],data['rating']
    # forward pass and loss
    y_predicted = model(user,item)
    preds = np.append(preds,y_predicted.flatten().detach().numpy())
    ratings = np.append(ratings,rating.flatten().detach().numpy())

    ustr = 'u' + str(user[0].item())
    if(user_dict.get(ustr) == None):
      user_dict[ustr] = []
      top_ratings[ustr] = set()
    user_dict[ustr].append((item[0].item(),y_predicted.flatten().detach().numpy()[0]))
    if(rating[0].item() >= 3):
      top_ratings[ustr].add(item[0].item())


#### RMSE





In [32]:
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(preds, ratings)
print(f'RMSE: {rmse:.4f}')

RMSE: 1.6221


#### Average Precision @ 3

In [108]:
p = 0
for key in user_dict:
  t3 = sorted(user_dict[key],reverse=True,key=lambda x: x[1])[:3]
  for x in t3:
    t1,t2 = x
    if(t1 in top_ratings[key]):
      p = p + 1
p3 = p/(len(user_dict)*3)
p3

0.830791575889615

### NCF

In [11]:
# From https://eigenvalue.medium.com/a-hackers-guide-to-neural-collaborative-filtering-with-pytorch-lightning-defa99236c78
class GMF(nn.Module):
  def __init__(self, num_users, num_items, embedding_dim=8):
    super(GMF, self).__init__()
    # Create embedding layers for users and items
    self.user_embedding = nn.Embedding(num_users, embedding_dim)
    self.item_embedding = nn.Embedding(num_items, embedding_dim)
    # Output layer to transform element-wise product
    self.output_layer = nn.Linear(embedding_dim, 16)

  def forward(self, user_ids, item_ids):
    # Get embeddings for users and items
    user_embedding = self.user_embedding(user_ids)
    item_embedding = self.item_embedding(item_ids)
    # Element-wise product
    gmf = torch.mul(user_embedding, item_embedding)
    # Transform through output layer and apply sigmoid
    output = torch.sigmoid(self.output_layer(gmf))
    return output

In [12]:
class MLP(nn.Module):
  def __init__(self, num_users, num_items, embedding_dim=8):
    super(MLP, self).__init__()
    # Create embedding layers for users and items
    self.U = nn.Embedding(num_users, embedding_dim)
    self.V = nn.Embedding(num_items, embedding_dim)
    # Output layer to transform element-wise product
    self.lin1 = nn.Linear(embedding_dim*2, 64)
    self.lin2 = nn.Linear(64, 16)
    self.relu = nn.ReLU()


  def forward(self, user_ids, item_ids):
    # Get embeddings for users and items
    u_u = self.U(user_ids)
    v_i = self.V(item_ids)
    x = torch.cat([u_u, v_i], dim=1)
    x = self.lin1(x)
    x = self.relu(x)
    x = self.lin2(x)

    return x

In [13]:
class NeuMF(nn.Module):
  def __init__(self, num_users, num_items, embedding_dim=8):
    super(NeuMF, self).__init__()
    # Create embedding layers for users and items
    self.gmf = GMF(num_users, num_items, embedding_dim)
    self.mlp = MLP(num_users, num_items, embedding_dim)
    self.lin1 = nn.Linear(32, 8)
    self.lin2 = nn.Linear(8, 1)
    self.relu = nn.ReLU()

  def forward(self, user_ids, item_ids):
    # Get embeddings for users and items
    gmf_out = self.gmf(user_ids,item_ids)
    mlp_out = self.mlp(user_ids,item_ids)
    x = torch.cat([gmf_out, mlp_out], dim=1)
    x = self.lin1(x)
    x = self.relu(x)
    x = self.lin2(x)

    return x

In [15]:
neumf = NeuMF(num_users,num_items)
learning_rate = 0.01
criterion_n = nn.MSELoss()
optimizer_n = torch.optim.Adam(neumf.parameters(), lr=learning_rate)

In [110]:
num_epochs = 30


neumf.train()

for epoch in range(num_epochs):
  for i,data in enumerate(trainiter):
      user,item,rating = data['user_id'],data['item_id'],data['rating']


      # forward pass and loss

      y_predicted = neumf.forward(user,item)

      loss = criterion_n(y_predicted,rating.unsqueeze(-1))

      # backward pass
      loss.backward()

      # updates
      optimizer_n.step()

      # zero gradients
      optimizer_n.zero_grad()

### Evaluation

In [115]:
preds = np.array([])
ratings = []
neumf.eval()
user_dict = dict()
top_ratings = dict()
for i,data in enumerate(testiter):
    user,item,rating = data['user_id'],data['item_id'],data['rating']
    # forward pass and loss
    y_predicted = neumf(user,item)
    preds = np.append(preds,y_predicted.flatten().detach().numpy())
    ratings = np.append(ratings,rating.flatten().detach().numpy())

    ustr = 'u' + str(user[0].item())
    if(user_dict.get(ustr) == None):
      user_dict[ustr] = []
      top_ratings[ustr] = set()
    user_dict[ustr].append((item[0].item(),y_predicted.flatten().detach().numpy()[0]))
    if(rating[0].item() >= 3):
      top_ratings[ustr].add(item[0].item())

#### RMSE

In [116]:
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(preds, ratings)
print(f'RMSE: {rmse:.4f}')

RMSE: 1.2490


#### Average Precision @ 3

In [117]:
p = 0
for key in user_dict:
  t3 = sorted(user_dict[key],reverse=True,key=lambda x: x[1])[:3]
  for x in t3:
    t1,t2 = x
    if(t1 in top_ratings[key]):
      p = p + 1
p3 = p/(len(user_dict)*3)
p3

0.8445896877269427

### Discussion

Matrix Factorization acheived an RMSE of 1.62 and an average precision @ 3 of 0.831 while NCF acheived an RMSE of 1.25 and an average precision @ 3 of 0.845. Thus, although NCF had slightly better prediction error across all ratings,  both models acheived similar performance when ranking the top three recomendations.

### Works Cited

Crivello, G. (2024, December 1). A hacker’s guide to neural collaborative filtering with Pytorch Lightning. Medium. https://eigenvalue.medium.com/a-hackers-guide-to-neural-collaborative-filtering-with-pytorch-lightning-defa99236c78

Murrell, T. (2023, February 7). Evaluating recommendation systems - precision@k, recall@k, and R-precision. Shaped. https://www.shaped.ai/blog/evaluating-recommendation-systems-part-1

Zhang, A., Lipton, Z., Li, M., & Smola, A. (2023). Dive into Deep Learning. Cambridge University Press.